# Extract Last Row Per Day - QQQ Options 5m

This notebook extracts the last row from each daily file of QQQ options 5m data.
Uses ThetaSyncManager internal methods to locate and read files.

In [1]:
import sys
import os
import pandas as pd
from pathlib import Path

# Add tdSynchManager to path
sys.path.insert(0, r'd:\Dropbox\TRADING\DATA FEEDERS AND APIS\ThetaData\tdSynchManager\src')

from tdSynchManager.manager import ThetaSyncManager
from tdSynchManager.config import ManagerConfig

In [2]:
# Create minimal config
cfg = ManagerConfig(
    root_dir=r"C:\Users\Federico\Downloads",
    max_concurrency=1
)

# Create manager instance (no client needed for reading files)
manager = ThetaSyncManager(cfg, client=None)

[2025-12-17 07:32:20.642] [VERSION] tdSynchManager shell v3 (EOD timestamp fix v1.0.9)


In [3]:
# Use manager's internal method to list files - check correct signature first
import inspect
sig = inspect.signature(manager._list_series_files)
print(f"Method signature: {sig}")
print(f"\nParameters:")
for param_name, param in sig.parameters.items():
    print(f"  {param_name}: {param.annotation if param.annotation != inspect.Parameter.empty else 'no type'}")

[2025-12-17 07:32:20.658] Method signature: (asset: 'str', symbol: 'str', interval: 'str', sink_lower: 'str') -> 'list'
[2025-12-17 07:32:20.658] 
Parameters:
[2025-12-17 07:32:20.658]   asset: str
[2025-12-17 07:32:20.658]   symbol: str
[2025-12-17 07:32:20.658]   interval: str
[2025-12-17 07:32:20.658]   sink_lower: str


In [4]:
# List all CSV files for QQQ options 5m
files = manager._list_series_files(
    asset="option",
    symbol="QQQ",
    interval="5m",
    sink_lower="csv"  # Must be "csv" or "parquet"
)

print(f"Found {len(files)} files for QQQ options 5m CSV")
print(f"\nFirst 5 files:")
for f in files[:5]:
    print(f"  {os.path.basename(f)}")

[2025-12-17 07:32:20.676] Found 46 files for QQQ options 5m CSV
[2025-12-17 07:32:20.676] 
First 5 files:
[2025-12-17 07:32:20.677]   2025-12-08T00-00-00Z-QQQ-option-5m_part01.csv
[2025-12-17 07:32:20.677]   2025-12-08T00-00-00Z-QQQ-option-5m_part02.csv
[2025-12-17 07:32:20.677]   2025-12-08T00-00-00Z-QQQ-option-5m_part03.csv
[2025-12-17 07:32:20.677]   2025-12-08T00-00-00Z-QQQ-option-5m_part04.csv
[2025-12-17 07:32:20.677]   2025-12-08T00-00-00Z-QQQ-option-5m_part05.csv


In [5]:
# Show available columns in CSV files
if files:
    sample_file = files[0]
    sample_df = pd.read_csv(sample_file)
    print(f"CSV columns ({len(sample_df.columns)}):")
    for i, col in enumerate(sample_df.columns, 1):
        print(f"  {i:2d}. {col}")
else:
    print("No files found!")

[2025-12-17 07:32:21.057] CSV columns (45):
[2025-12-17 07:32:21.057]    1. symbol
[2025-12-17 07:32:21.058]    2. expiration
[2025-12-17 07:32:21.058]    3. strike
[2025-12-17 07:32:21.058]    4. right
[2025-12-17 07:32:21.058]    5. timestamp
[2025-12-17 07:32:21.058]    6. open
[2025-12-17 07:32:21.058]    7. high
[2025-12-17 07:32:21.058]    8. low
[2025-12-17 07:32:21.058]    9. close
[2025-12-17 07:32:21.058]   10. volume
[2025-12-17 07:32:21.058]   11. count
[2025-12-17 07:32:21.058]   12. vwap
[2025-12-17 07:32:21.058]   13. bid
[2025-12-17 07:32:21.058]   14. ask
[2025-12-17 07:32:21.058]   15. delta
[2025-12-17 07:32:21.058]   16. theta
[2025-12-17 07:32:21.058]   17. vega
[2025-12-17 07:32:21.058]   18. rho
[2025-12-17 07:32:21.058]   19. epsilon
[2025-12-17 07:32:21.058]   20. lambda
[2025-12-17 07:32:21.058]   21. gamma
[2025-12-17 07:32:21.058]   22. vanna
[2025-12-17 07:32:21.058]   23. charm
[2025-12-17 07:32:21.058]   24. vomma
[2025-12-17 07:32:21.058]   25. veta
[202

In [6]:
# Extract 2 rows per DAY (1 call + 1 put) with filters:
# - Expiration = next trading day (1DTE) - skips weekends
# - Last rows before market close
# - Strike closest to underlying_price

from datetime import timedelta

def get_next_trading_day(date):
    """Calculate next trading day, skipping weekends."""
    next_day = date + timedelta(days=1)
    
    # If Saturday (5), add 2 days to get to Monday
    if next_day.weekday() == 5:
        next_day += timedelta(days=2)
    # If Sunday (6), add 1 day to get to Monday
    elif next_day.weekday() == 6:
        next_day += timedelta(days=1)
    
    return next_day.strftime('%Y-%m-%d')

# Group files by date
files_by_date = {}
for file_path in files:
    filename = os.path.basename(file_path)
    date_str = filename.split('T')[0]
    if date_str not in files_by_date:
        files_by_date[date_str] = []
    files_by_date[date_str].append(file_path)

print(f"Found {len(files_by_date)} unique days")

extracted_rows = []

for date_str in sorted(files_by_date.keys()):
    try:
        file_date = pd.to_datetime(date_str)
        # Calculate next TRADING day (skip weekends)
        next_day = get_next_trading_day(file_date)
        
        print(f"  {date_str} ({file_date.strftime('%A')}): looking for exp={next_day}", end="")
        
        # Read ALL files for this day and concatenate
        day_dfs = []
        for file_path in files_by_date[date_str]:
            df = pd.read_csv(file_path)
            if not df.empty:
                day_dfs.append(df)
        
        if not day_dfs:
            print(f" -> No data")
            continue
        
        # Concatenate all parts
        full_day_df = pd.concat(day_dfs, ignore_index=True)
        full_day_df['timestamp'] = pd.to_datetime(full_day_df['timestamp'])
        
        # Get last timestamp (before market close)
        last_timestamp = full_day_df['timestamp'].max()
        last_df = full_day_df[full_day_df['timestamp'] == last_timestamp].copy()
        
        # Filter: expiration = next trading day (1DTE)
        last_df = last_df[last_df['expiration'] == next_day]
        
        if last_df.empty:
            print(f" -> No 1DTE contracts")
            continue
        
        # Get underlying price
        if 'underlying_price' not in last_df.columns or last_df['underlying_price'].isna().all():
            print(f" -> Missing underlying_price")
            continue
        
        underlying_price = last_df['underlying_price'].iloc[0]
        
        # Extract 1 call + 1 put (closest to underlying)
        for right in ['call', 'put']:
            right_df = last_df[last_df['right'] == right].copy()
            
            if right_df.empty:
                print(f" -> No {right} contracts")
                continue
            
            # Find strike closest to underlying_price
            right_df['distance'] = abs(right_df['strike'] - underlying_price)
            
            # Get strikes just below and above
            strikes_below = right_df[right_df['strike'] <= underlying_price]
            strikes_above = right_df[right_df['strike'] > underlying_price]
            
            # Pick the closest one
            closest_row = None
            if not strikes_below.empty and not strikes_above.empty:
                closest_below = strikes_below.loc[strikes_below['distance'].idxmin()]
                closest_above = strikes_above.loc[strikes_above['distance'].idxmin()]
                if closest_below['distance'] <= closest_above['distance']:
                    closest_row = closest_below
                else:
                    closest_row = closest_above
            elif not strikes_below.empty:
                closest_row = strikes_below.loc[strikes_below['distance'].idxmin()]
            elif not strikes_above.empty:
                closest_row = strikes_above.loc[strikes_above['distance'].idxmin()]
            
            if closest_row is not None:
                # Calculate mid price
                mid_price = (closest_row['bid'] + closest_row['ask']) / 2 if pd.notna(closest_row['bid']) and pd.notna(closest_row['ask']) else None
                
                extracted_rows.append({
                    'date': date_str,
                    'timestamp': closest_row['timestamp'],
                    'expiration': closest_row['expiration'],
                    'strike': closest_row['strike'],
                    'right': right,
                    'open': closest_row['open'],
                    'high': closest_row['high'],
                    'low': closest_row['low'],
                    'close': closest_row['close'],
                    'bid': closest_row['bid'],
                    'ask': closest_row['ask'],
                    'mid_price': mid_price,
                    'underlying_price': underlying_price
                })
        
        print(f" -> OK (strike={closest_row['strike']}, underlying={underlying_price:.2f})")
                
    except Exception as e:
        print(f" -> Error: {e}")
        import traceback
        traceback.print_exc()

print(f"\n=== EXTRACTION COMPLETE ===")
print(f"Total rows extracted: {len(extracted_rows)}")

# Create DataFrame with extracted data
options_df = pd.DataFrame(extracted_rows)

if not options_df.empty:
    options_df = options_df.sort_values(['date', 'right'])
    print(f"\n=== EXTRACTED OPTIONS DATA ===")
    print(f"Days: {options_df['date'].nunique()}")
    print(f"Date range: {options_df['date'].min()} to {options_df['date'].max()}")
    display(options_df)
else:
    print("No data extracted!")

[2025-12-17 07:32:21.098] Found 7 unique days
[2025-12-17 07:32:21.102]   2025-12-08 (Monday): looking for exp=2025-12-09[2025-12-17 07:32:23.376]  -> OK (strike=624.0, underlying=624.19)
[2025-12-17 07:32:23.376]   2025-12-09 (Tuesday): looking for exp=2025-12-10[2025-12-17 07:32:25.491]  -> OK (strike=625.0, underlying=625.11)
[2025-12-17 07:32:25.491]   2025-12-10 (Wednesday): looking for exp=2025-12-11[2025-12-17 07:32:27.707]  -> OK (strike=628.0, underlying=627.53)
[2025-12-17 07:32:27.707]   2025-12-11 (Thursday): looking for exp=2025-12-12[2025-12-17 07:32:30.424]  -> OK (strike=626.0, underlying=625.60)
[2025-12-17 07:32:30.425]   2025-12-12 (Friday): looking for exp=2025-12-15[2025-12-17 07:32:33.104]  -> OK (strike=614.0, underlying=613.57)
[2025-12-17 07:32:33.114]   2025-12-15 (Monday): looking for exp=2025-12-16[2025-12-17 07:32:35.384]  -> OK (strike=610.0, underlying=610.49)
[2025-12-17 07:32:35.384]   2025-12-16 (Tuesday): looking for exp=2025-12-17[2025-12-17 07:32:38

,date,timestamp,expiration,strike,right,open,high,low,close,bid,ask,mid_price,underlying_price
0,2025-12-08,2025-12-08 21:00:00+00:00,2025-12-09,624.0,call,0.0,0.0,0.0,0.0,2.03,2.05,2.040,624.19
1,2025-12-08,2025-12-08 21:00:00+00:00,2025-12-09,624.0,put,0.0,0.0,0.0,0.0,1.77,1.81,1.790,624.19
2,2025-12-09,2025-12-09 21:00:00+00:00,2025-12-10,625.0,call,0.0,0.0,0.0,0.0,2.72,2.74,2.730,625.11
3,2025-12-09,2025-12-09 21:00:00+00:00,2025-12-10,625.0,put,0.0,0.0,0.0,0.0,2.54,2.58,2.560,625.11
4,2025-12-10,2025-12-10 21:00:00+00:00,2025-12-11,628.0,call,0.0,0.0,0.0,0.0,2.45,2.50,2.475,627.53
5,2025-12-10,2025-12-10 21:00:00+00:00,2025-12-11,628.0,put,0.0,0.0,0.0,0.0,2.83,2.87,2.850,627.53
6,2025-12-11,2025-12-11 21:00:00+00:00,2025-12-12,626.0,call,0.0,0.0,0.0,0.0,2.25,2.28,2.265,625.60
7,2025-12-11,2025-12-11 21:00:00+00:00,2025-12-12,626.0,put,0.0,0.0,0.0,0.0,2.49,2.54,2.515,625.60
8,2025-12-12,2025-12-12 21:00:00+00:00,2025-12-15,614.0,call,0.0,0.0,0.0,0.0,2.57,2.62,2.595,613.57
9,2025-12-12,2025-12-12 21:00:00+00:00,2025-12-15,614.0,put,0.0,0.0,0.0,0.0,2.88,2.95,2.915,613.57


In [7]:
# Calculate Support and Resistance for each day
if not options_df.empty:
    sr_rows = []
    
    for date in options_df['date'].unique():
        day_data = options_df[options_df['date'] == date]
        
        call_data = day_data[day_data['right'] == 'call']
        put_data = day_data[day_data['right'] == 'put']
        
        if not call_data.empty and not put_data.empty:
            call_row = call_data.iloc[0]
            put_row = put_data.iloc[0]
            
            strike = call_row['strike']
            mid_call = call_row['mid_price']
            mid_put = put_row['mid_price']
            
            if pd.notna(mid_call) and pd.notna(mid_put):
                resistance = strike + mid_call + mid_put
                support = strike - mid_call - mid_put
                
                sr_rows.append({
                    'date': date,
                    'timestamp': call_row['timestamp'],
                    'underlying_price': call_row['underlying_price'],
                    'strike': strike,
                    'mid_call': mid_call,
                    'mid_put': mid_put,
                    'support': support,
                    'resistance': resistance,
                    'range': resistance - support
                })
    
    # Create DataFrame
    sr_df = pd.DataFrame(sr_rows)
    
    if not sr_df.empty:
        print(f"\n=== SUPPORT & RESISTANCE LEVELS ===")
        print(f"Days: {len(sr_df)}")
        display(sr_df)
        
        # Save both DataFrames to CSV
        options_df.to_csv("qqq_options_1dte.csv", index=False)
        sr_df.to_csv("qqq_support_resistance.csv", index=False)
        print(f"\nSaved:")
        print(f"  - qqq_options_1dte.csv ({len(options_df)} rows)")
        print(f"  - qqq_support_resistance.csv ({len(sr_df)} rows)")
    else:
        print("No support/resistance data calculated!")
else:
    print("No options data available!")

[2025-12-17 07:32:38.172] 
=== SUPPORT & RESISTANCE LEVELS ===
[2025-12-17 07:32:38.173] Days: 7


,date,timestamp,underlying_price,strike,mid_call,mid_put,support,resistance,range
0,2025-12-08,2025-12-08 21:00:00+00:00,624.19,624.0,2.040,1.790,620.170,627.830,7.66
1,2025-12-09,2025-12-09 21:00:00+00:00,625.11,625.0,2.730,2.560,619.710,630.290,10.58
2,2025-12-10,2025-12-10 21:00:00+00:00,627.53,628.0,2.475,2.850,622.675,633.325,10.65
3,2025-12-11,2025-12-11 21:00:00+00:00,625.60,626.0,2.265,2.515,621.220,630.780,9.56
4,2025-12-12,2025-12-12 21:00:00+00:00,613.57,614.0,2.595,2.915,608.490,619.510,11.02
5,2025-12-15,2025-12-15 21:00:00+00:00,610.49,610.0,2.710,2.170,605.120,614.880,9.76
6,2025-12-16,2025-12-16 21:00:00+00:00,611.80,612.0,2.220,2.345,607.435,616.565,9.13


[2025-12-17 07:32:38.195] 
Saved:
[2025-12-17 07:32:38.196]   - qqq_options_1dte.csv (14 rows)
[2025-12-17 07:32:38.196]   - qqq_support_resistance.csv (7 rows)
